<a href="https://colab.research.google.com/github/niaza64/FriedrichNietzscheSummary/blob/main/NietzchePhilosphySummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch

Installing some dependencies

In [ ]:
import torch

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:

filename = 'Data_BGE.txt'

with open(filename, 'r', encoding='utf-8') as file:
    nietzsche_text = file.read()


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

# Now you can tokenize your text with padding
inputs = tokenizer(nietzsche_text, return_tensors="pt", truncation=True, padding=True)

In [ ]:
from torch.utils.data import DataLoader

# DataLoader can be used for managing batches
train_data = DataLoader(inputs, batch_size=8)

In [ ]:
print(train_data)

In [ ]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        """
        Returns input and label (input shifted to the right by one token).
        """
        input_ids = self.encodings.input_ids[idx]
        attention_mask = self.encodings.attention_mask[idx]

        # Create labels by shifting the input_ids to the right
        labels = input_ids.clone()
        labels[:-1] = input_ids[1:]

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


    def __len__(self):
        # Return the length of the input_ids
        return len(self.encodings.input_ids)

# Create a dataset instance
dataset = TextDataset(inputs)

# Use the custom dataset with DataLoader
train_data = DataLoader(dataset, batch_size=8)

In [ ]:
for batch in train_data:
    print(batch)
    break

{'input_ids': tensor([[ 171,  119,  123,  ..., 8104,  878,  514]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[119, 123,  47,  ..., 878, 514, 514]])}


In [ ]:
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
for epoch in range(100):  # Number of epochs
    for batch in train_data:
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 0.055209752172231674
Epoch: 1, Loss: 0.06731650233268738
Epoch: 2, Loss: 0.07287133485078812
Epoch: 3, Loss: 0.08923991024494171
Epoch: 4, Loss: 0.07615947723388672
Epoch: 5, Loss: 0.07746896147727966
Epoch: 6, Loss: 0.06533140689134598
Epoch: 7, Loss: 0.07956995815038681
Epoch: 8, Loss: 0.06764628738164902
Epoch: 9, Loss: 0.08529496192932129
Epoch: 10, Loss: 0.05933871492743492
Epoch: 11, Loss: 0.08055520802736282
Epoch: 12, Loss: 0.06270216405391693
Epoch: 13, Loss: 0.07725837826728821
Epoch: 14, Loss: 0.06790415942668915
Epoch: 15, Loss: 0.052547551691532135
Epoch: 16, Loss: 0.051172174513339996
Epoch: 17, Loss: 0.04448238015174866
Epoch: 18, Loss: 0.06800917536020279
Epoch: 19, Loss: 0.051617514342069626
Epoch: 20, Loss: 0.055400021374225616
Epoch: 21, Loss: 0.054217446595430374
Epoch: 22, Loss: 0.040332283824682236
Epoch: 23, Loss: 0.04214561730623245
Epoch: 24, Loss: 0.049551207572221756
Epoch: 25, Loss: 0.04823623225092888
Epoch: 26, Loss: 0.041334688663482666
Ep

In [ ]:
# Saving to Colab's local storage
model.save_pretrained("/content/my_model")
tokenizer.save_pretrained("/content/my_model")


('/content/my_model/tokenizer_config.json',
 '/content/my_model/special_tokens_map.json',
 '/content/my_model/vocab.json',
 '/content/my_model/merges.txt',
 '/content/my_model/added_tokens.json')

In [ ]:
model.eval()  # Set the model to evaluation mode

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
prompt = "What is will to truth"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [ ]:
output = model.generate(input_ids, max_length=200)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is will to truth the of, and who is to judge? The of, and who is to judge? The philosophers of, have a subject, for they not only speak, they also write, and understand, but also to and judge. Socrates, is Truth the of, speak, and judge, speak the, the is the, the, the, the, the; the is the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the


In [ ]:
prompt = "What is will to truth"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate attention mask (1s for each token in the input_ids)
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

output = model.generate(
    input_ids=input_ids,
    max_length=200,
    temperature=0.9,  # Adjust between 0.7 and 1.0 to see the effect
    top_k=50,         # Try different values like 20, 40, 50
    top_p=0.9,        # Nucleus sampling, values typically between 0.8 and 0.95
    repetition_penalty=1.2,  # Adjust as necessary
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(output[0], skip_special_tokens=True))


What is will to truth the of, and who knows? Will Truth bete philosophers a forgery or anc it; they that which pret in form speak good only when spoken with Good can neither bring nor maintain themselves ground: so philosophy their lies all things are unsely as matter itself sad at heart But find again strength by diligence what kind art patience Spirit bestows on scoff whom It worst lest wrong belief fost Worst denial stands still Let not unfulfulness seriously discouraged Grow popular opinion Ist really founded whatever although certainly noble Certainly dangerous even wicked And tires questions enormous Theories verymat unsound indeed Fornication error its branches philosophical controversy present day Socialism many tempts attempts deception Science superstitionwhich has pique our awe Errors spirits distress ast such have no control over. us we draw courage from facts vain claims Pure doctrine too often turns ugly Its aim great usually ends up dogmatic—admitting fault spoke earlier

In [106]:
from ipywidgets import widgets, Layout
from IPython.display import display


prompt_input = widgets.Textarea(
    placeholder='Type something',
    description='Prompt:',
    layout=Layout(width='100%', height='100px')
)


generate_button = widgets.Button(
    description='Generate',
    button_style='info',  # 'success', 'info', 'warning', 'danger', or ''
    layout=Layout(width='100px', margin='0px 0px 0px 10px')
)

output_widget = widgets.Output()


def on_generate_button_clicked(b):

    output_widget.clear_output()

    with output_widget:
        input_ids = tokenizer(prompt_input.value, return_tensors="pt").input_ids

        output = model.generate(
            input_ids=input_ids,
            max_length=200,
            temperature=0.9,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the generated sequence and print it
        generated_sequence = tokenizer.decode(output[0], skip_special_tokens=True)
        print(generated_sequence)

generate_button.on_click(on_generate_button_clicked)

display(prompt_input, generate_button, output_widget)


Textarea(value='', description='Prompt:', layout=Layout(height='100px', width='100%'), placeholder='Type somet…

Button(button_style='info', description='Generate', layout=Layout(margin='0px 0px 0px 10px', width='100px'), s…

Output()